In [180]:
#import statements
import pandas as pd 
import numpy as np 
import pickle
pd.set_option('display.max_columns', None)

Read in data   
Read in those properties that fall under "Multi Family" and "Single Family"

**Find the data codebook here**:   https://metadata.phila.gov/#home/datasetdetails/5543865f20583086178c4ee5/representationdetails/55d624fdad35c7e854cb21a4/?view_287_page=1

**Columns to keep**:  'assessment_date', 'basements', 'building_code_description',
                 'category_code_description','census_tract', 'central_air','depth',
                 'exterior_condition', 'fireplaces', 'frontage', 'fuel', 'garage_spaces',
                 'geographic_ward', 'house_number', 
                 'interior_condition',
                 'market_value', 'market_value_date', 'number_of_bathrooms', 
                 'number_of_bedrooms', 'number_of_rooms','number_stories', 'quality_grade', 
                 'sale_date', 'sale_price', 'street_designation',
                 'topography', 'total_area','total_livable_area', 'type_heater', 
                 'unfinished',  'view_type', 'year_built',
                 'zip_code',  'lat', 'lng'

In [160]:
#place columns of interest into a list
cols_interest = ['assessment_date', 'basements', 
                 'category_code_description','census_tract', 'central_air','depth',
                 'exterior_condition', 'fireplaces', 'frontage', 'fuel', 'garage_spaces',
                 'geographic_ward', 'house_number', 
                 'interior_condition',
                 'market_value', 'market_value_date', 'number_of_bathrooms', 
                 'number_of_bedrooms', 'number_of_rooms','number_stories', 'quality_grade', 
                 'sale_date', 'sale_price', 'street_designation',
                 'topography', 'total_area','total_livable_area', 'type_heater', 
                 'unfinished',  'view_type', 'year_built',
                 'zip_code',  'lat', 'lng']

In [161]:
#read in data using columns of interest 
#query only those multifamily and single family homes
dat = (pd.read_csv("/home/jovyan/work/Philadelphia-Housing/processing/opa_properties_public.csv", 
                 usecols = cols_interest)
       .query('category_code_description == "Multi Family" | category_code_description == "Single Family"')
)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (67,71) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


We re-code four columns of interest (basements, central_air, fuel, topography, type_heater).    

In [162]:
dat['basements'] = dat['basements'].map({'A': 'full', 'B': 'full','C': 'full','D': 'full','I': 'full',
                                        'E': 'partial','F': 'partial','G': 'partial','H': 'partial','J': 'partial',
                                        '0': 'None'})
dat['central_air'] = dat['central_air'].map({'0': 'N', '1': 'Y','Y': 'Y', 'N': 'N'})

dat['fuel'] = dat['fuel'].map({'A': 'NG', 'B': 'Oil','C': 'Electric',
                              'E': 'other', 'G': 'other','H': 'other','I':'other'})
dat['topography'] = dat['topography'].map({'A': 'A', 'B': 'B','C': 'C',
                              'D': 'D', 'E': 'E','F': 'F'})

dat['type_heater'] = dat['type_heater'].map({'A': 'A', 'B': 'B','C': 'C',
                              'D': 'D', 'E': 'E','G': 'G','H': 'H'})


# dat['street_designation'] = dat['street_designation'].map({'ST': 'STAAA',
#                                                            'AVE': 'AVE',
#                                                            'RD': 'RD',
#                                                            'LA': 'LA',
#                                                            'DR': 'DR',
#                                                            'PL': 'PL',
#                                                            #'SQ': 'SQAAAA',
#                                                            #'ST': 'ST',
#                                                            #'AVE': 'AVE',
#                                                            #'RD': 'RD',
#                                                            #'LA': 'LA',
#                                                            #'DR': 'DR',
#                                                            #'PL': 'PL',
#                                                            'PL': 'PL' })

dat['street_designation'] = dat['street_designation'].astype("string")

# dat['street_designation'] = np.where( dat['street_designation'] == "ST", "ST",
#                                   (np.where( dat['street_designation'] == "AVE", "AVE","Other")))


# ST      177010
# AVE      32703
# RD       12143
# LA        3139
# DR        2487
# PL        2288
# SQ        1288
# BLV       1001
# CT         775
# TER        716
# WAY        659
# LN         526
# CIR        348
# PK         164
# PKY        117
# BLVD       108
# PLZ         66
# MEW         33
# ALY         27
# PIKE        25
# WLK         14
# PKWY         9
# MEWS         7
# ROW          4
# WALK         4
# HTS          3
# PTH          2
# PATH         2

In [139]:
dat['street_designation']

0         ST  
1         ST  
2         AVE 
3         AVE 
9         ST  
          ... 
581405    ST  
581406    AVE 
581407    ST  
581409    ST  
581410    ST  
Name: street_designation, Length: 504500, dtype: string

In [140]:
np.where( np.array(dat['street_designation']) == "ST", "ST",
         (np.where( np.array(dat['street_designation']) == "AVE", "AVE","Other")))

array(['Other', 'Other', 'Other', ..., 'Other', 'Other', 'Other'],
      dtype='<U5')

In [129]:
dat['street_designation'].value_counts()

Other    504500
Name: street_designation, dtype: int64

Below, we output the percentage missing by column:  
Note that assessment_date, fuel, market_value_date, quality_grade, and unfinished are all over ~90% missing.   
For these reasons, we will drop these columns from the analysis, as they will not provide robust and useful information for the analysis and prediction process.  

In [145]:
dat.isnull().sum() * 100 / len(dat)

assessment_date               93.949851
basements                     36.815857
category_code_description      0.000000
census_tract                   0.006938
central_air                   44.558375
depth                          0.077899
exterior_condition             0.088206
fireplaces                     0.164718
frontage                       0.079088
fuel                          97.111992
garage_spaces                  0.199405
geographic_ward                0.006938
house_number                   0.000000
interior_condition             0.109812
market_value                   0.018236
market_value_date            100.000000
number_of_bathrooms            0.143707
number_of_bedrooms             0.083845
number_of_rooms                5.908622
number_stories                 0.083251
quality_grade                 89.448167
sale_date                      0.001586
sale_price                     0.002180
street_designation             0.000000
topography                     6.621011


In [163]:
dat = dat.drop(['assessment_date', 'fuel', 
         'market_value_date', 'quality_grade', 
         'unfinished', 'central_air', 'type_heater',
               'street_designation'], axis = 1)

Next, we will remove those datapoints/rows that do not have values for **sale price** or **sale date**.   
These are removed since these are our outcome variables.  
Both the sales prices and the date which the house was sold is necessary to perform our experiment.  

In [164]:
#drop the rows which 'sale_date' is NAN
dat = dat[dat['sale_date'].notna()]
#drop the rows which 'sale_price' is NAN
dat = dat[dat['sale_price'].notna()]

Find the percentage of homes that have a sales price of greater than $5  

In [165]:
(dat['sale_price'] > 5_000_000).mean() *100

0.2820675971131184

*Note*: 0.28% of the data has a sale_price of greater than $5 million.   
We remove these from the dataset since we are not interested in these homes.  

In [166]:
dat = dat[dat['sale_price'] < 5_000_000]

Next, we deal with dates:    
First we coerce the dates    
Next we will break apart the date elements into variables   

In [167]:
#create a date variable
dat['sale_date'] = pd.to_datetime(dat['sale_date'])
#break apart the date to new variables:  
dat['sale_year'] = dat['sale_date'].dt.year
dat['sale_month'] = dat['sale_date'].dt.month
dat['sale_week'] = dat['sale_date'].dt.isocalendar().week
dat['sale_day'] = dat['sale_date'].dt.day
dat['sale_dow'] = dat['sale_date'].dt.dayofweek
#change the sale week column type to int since it is coerced originally to 
#UInt32
dat['sale_week'] = dat['sale_week'].astype(int)

Certain 'year_built' inputs are incorrect.    
We ensure to change these below:   

In [168]:
dat['year_built'] = dat['year_built'].replace('196Y', np.NaN)
#convert to numeric
dat["year_built"] = pd.to_numeric(dat["year_built"])
#replace year 0 with NaN
dat['year_built'] = dat['year_built'].replace(0, np.NaN)

In [169]:
dat['year_built'].describe(include = 'all')

count    502272.000000
mean       1937.034941
std          27.619292
min        1652.000000
25%        1920.000000
50%        1925.000000
75%        1950.000000
max        2022.000000
Name: year_built, dtype: float64

In [170]:
dat['year_built'].isnull().sum() * 100 / len(dat)

0.1528702431809733

We will now perform preliminary train/test processing of the data necessary for modeling    

### Training / Test Split

First, we will split the data into training and test sets:   
The training dataset contains all housing prices that were sold during the years of 2010 - 2019.    
The test dataset contains all housing prices that were sold during 2020 and 2021.  

In [171]:
#first subset the data only to 2010 - 2021
dat = dat[dat['sale_year'] > 2009]
#split to the training set 
train = dat[dat['sale_year'] < 2020]
#split to the test set 
test = dat[dat['sale_year'] > 2019]

#finally, drop the date column:  
train = train.drop(['sale_date'], axis=1)
test = test.drop(['sale_date'], axis=1)

In [189]:
dat.head()

,basements,category_code_description,census_tract,depth,exterior_condition,fireplaces,frontage,garage_spaces,geographic_ward,house_number,interior_condition,market_value,number_of_bathrooms,number_of_bedrooms,number_of_rooms,number_stories,sale_date,sale_price,topography,total_area,total_livable_area,view_type,year_built,zip_code,lat,lng,sale_year,sale_month,sale_week,sale_day,sale_dow
0,full,Single Family,139.0,94.0,1.0,NaN,16.0,NaN,29.0,2334,1.0,263000.0,NaN,3.0,10.0,1.0,2021-08-06,263000.0,F,1542.0,1242.0,NaN,2019.0,19121.0,-75.174210,39.979126,2021,8,31,6,4
1,None,Single Family,8.0,0.0,3.0,0.0,0.0,0.0,8.0,219,3.0,309800.0,0.0,0.0,NaN,1.0,2021-07-16,272500.0,F,0.0,524.0,C,1900.0,19103.0,-75.170423,39.949254,2021,7,28,16,4
2,NaN,Single Family,160.0,50.0,1.0,NaN,NaN,NaN,31.0,2216,1.0,660000.0,2.0,3.0,NaN,3.0,2021-07-06,655000.0,F,800.0,2499.0,I,2020.0,19125.0,-75.128893,39.978672,2021,7,27,6,1
3,full,Single Family,216.0,109.0,1.0,0.0,80.0,NaN,21.0,7354,1.0,570400.0,2.0,3.0,6.0,2.0,2021-08-10,605950.0,F,8219.0,2716.0,I,2021.0,19128.0,-75.235043,40.045622,2021,8,32,10,1
9,NaN,Multi Family,215.0,38.0,3.0,0.0,68.0,0.0,21.0,4620,3.0,202600.0,0.0,0.0,NaN,2.0,2021-11-09,1.0,F,4712.0,2356.0,I,1940.0,19127.0,-75.229891,40.029924,2021,11,45,9,1


We pickle the training data that has not yet been processed for visualization purposes

In [36]:
pickle_out = open("housedat_train_vis.pickle","wb")
pickle.dump(train, pickle_out)
pickle_out.close()

#### Processing part 2
Next, we process the training data via one hot encoding

In [173]:
# convert categorical variables to dummy variables and add dummy variables to data frame
cat_vars = train[['basements','category_code_description',
               'topography', 'view_type']]
cat_dummies = pd.get_dummies(cat_vars, drop_first=True)
train = train.drop(['basements','category_code_description',
               'topography', 'view_type'], axis=1)
train = pd.concat([train, cat_dummies], axis=1)

In [174]:
train.head()

,census_tract,depth,exterior_condition,fireplaces,frontage,garage_spaces,geographic_ward,house_number,interior_condition,market_value,number_of_bathrooms,number_of_bedrooms,number_of_rooms,number_stories,sale_price,total_area,total_livable_area,year_built,zip_code,lat,lng,sale_year,sale_month,sale_week,sale_day,sale_dow,basements_full,basements_partial,category_code_description_Single Family,topography_B,topography_C,topography_D,topography_E,topography_F,view_type_A,view_type_B,view_type_C,view_type_D,view_type_E,view_type_H,view_type_I
160,114.0,97.0,4.0,0.0,16.0,0.0,34.0,1612,4.0,77300.0,1.0,3.0,NaN,2.0,1.0,1496.0,1250.0,1925.0,19151.0,-75.244809,39.975755,2019,7,27,3,2,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1
222,265.0,83.0,4.0,0.0,18.0,1.0,10.0,6606,4.0,116200.0,1.0,3.0,NaN,2.0,1.0,1493.0,1188.0,1920.0,19138.0,-75.158656,40.057118,2015,7,27,1,2,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1
346,71.0,59.0,4.0,0.0,14.0,0.0,51.0,5746,4.0,58300.0,1.0,3.0,NaN,2.0,1.0,838.0,962.0,1925.0,19143.0,-75.230115,39.937673,2015,7,29,13,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1
890,107.0,75.0,4.0,0.0,15.0,0.0,6.0,4220,4.0,46400.0,0.0,0.0,NaN,2.0,40000.0,1100.0,1214.0,1920.0,19104.0,-75.209270,39.971367,2018,6,24,12,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1
937,312.0,68.0,4.0,0.0,19.0,1.0,54.0,1447,4.0,109900.0,1.0,3.0,NaN,2.0,1.0,1313.0,1166.0,1950.0,19149.0,-75.079455,40.035077,2017,3,12,24,4,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1


In [175]:
train = train.fillna(train.mean())

In [176]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(train.drop(['sale_price'], axis=1),
           train['sale_price'])
reg.score(train.drop(['sale_price'], axis=1),
           train['sale_price'])

0.2154793740745844

In [179]:
import statsmodels.api as sm
model = sm.OLS(train['sale_price'], 
               train.drop(['sale_price'], axis=1)
           )
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             sale_price   R-squared (uncentered):                   0.396
Model:                            OLS   Adj. R-squared (uncentered):              0.395
Method:                 Least Squares   F-statistic:                              3241.
Date:                Wed, 06 Apr 2022   Prob (F-statistic):                        0.00
Time:                        16:05:53   Log-Likelihood:                     -2.7611e+06
No. Observations:              198168   AIC:                                  5.522e+06
Df Residuals:                  198128   BIC:                                  5.523e+06
Df Model:                          40                                                  
Covariance Type:            nonrobust                                                  
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
census_tract                             -499.4182     10.428    -47.893      0.000    -519.856    -478.980
depth                                      -3.2617      0.485     -6.725      0.000      -4.212      -2.311
exterior_condition                      -5588.8538   3290.294     -1.699      0.089    -1.2e+04     860.043
fireplaces                               1.201e+05   2327.336     51.597      0.000    1.16e+05    1.25e+05
frontage                                    0.3144      0.060      5.255      0.000       0.197       0.432
garage_spaces                            1.623e+04    797.018     20.365      0.000    1.47e+04    1.78e+04
geographic_ward                         -1001.9637     41.187    -24.327      0.000   -1082.689    -921.239
house_number                               -2.6794      0.214    -12.531      0.000      -3.099      -2.260
interior_condition                      -6.183e+04   3269.041    -18.913      0.000   -6.82e+04   -5.54e+04
market_value                                0.0069      0.001     11.967      0.000       0.006       0.008
number_of_bathrooms                      3.636e+04   1086.436     33.467      0.000    3.42e+04    3.85e+04
number_of_bedrooms                      -7112.0600    669.788    -10.618      0.000   -8424.828   -5799.292
number_of_rooms                         -8008.0953    330.419    -24.236      0.000   -8655.709   -7360.481
number_stories                           1.983e+04    348.912     56.833      0.000    1.91e+04    2.05e+04
total_area                                  1.4517      0.087     16.689      0.000       1.281       1.622
total_livable_area                         -0.3347      0.095     -3.513      0.000      -0.521      -0.148
year_built                                360.0478     24.164     14.900      0.000     312.686     407.409
zip_code                                 -954.1772     41.409    -23.043      0.000   -1035.338    -873.016
lat                                      2.307e+05   1.15e+04     20.030      0.000    2.08e+05    2.53e+05
lng                                      3.638e+05    1.8e+04     20.197      0.000    3.28e+05    3.99e+05
sale_year                                1.037e+04    215.277     48.165      0.000    9946.849    1.08e+04
sale_month                               -508.2432    847.238     -0.600      0.549   -2168.810    1152.324
sale_week                                 586.8117    194.357      3.019      0.003     205.877     967.746
sale_day                                 1190.2229     72.131     16.501      0.000    1048.847    1331.598
sale_dow                                -1120.9335    398.697     -2

In [55]:
import sklearn
from sklearn.linear_model import LinearRegression
import xgboost as xgb

data_dmatrix = xgb.DMatrix(train.drop(['sale_price'], axis=1),
            train['sale_price'])

xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)


In [57]:
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor

# clf = tree.DecisionTreeRegressor()
# clf = clf.fit(train.drop(['sale_price'], axis=1),
#             train['sale_price'])  

train = train.fillna(train.mean())
   
    
regr = RandomForestRegressor()
regr.fit(train.drop(['sale_price'], axis=1),
           train['sale_price'])

RandomForestRegressor()

In [59]:
from sklearn.metrics import mean_squared_error
rfpreds = regr.predict(train.drop(['sale_price'], axis=1))
mean_squared_error(train['sale_price'], rfpreds)

4659900398.913846

In [71]:
# convert categorical variables to dummy variables and add dummy variables to data frame
cat_vars = test[['basements','category_code_description','street_designation',
               'topography', 'view_type']]
cat_dummies = pd.get_dummies(cat_vars, drop_first=True)
test = test.drop(['basements','category_code_description','street_designation',
               'topography', 'view_type'], axis=1)
test = pd.concat([test, cat_dummies], axis=1)
test = test.fillna(test.mean())
rfpreds_test = regr.predict(test.drop(['sale_price'], axis=1))
mean_squared_error(test['sale_price'], rfpreds)

ValueError: Number of features of the model must match the input. Model n_features is 64 and input n_features is 66 

In [75]:
dat['street_designation'].value_counts()

ST      177010
AVE      32703
RD       12143
LA        3139
DR        2487
PL        2288
SQ        1288
BLV       1001
CT         775
TER        716
WAY        659
LN         526
CIR        348
PK         164
PKY        117
BLVD       108
PLZ         66
MEW         33
ALY         27
PIKE        25
WLK         14
PKWY         9
MEWS         7
ROW          4
WALK         4
HTS          3
PTH          2
PATH         2
Name: street_designation, dtype: int64

In [72]:
test.shape

(37500, 67)

In [73]:
train.shape

(198168, 65)

In [74]:
test[test.columns.difference(train.columns)]

,street_designation_MEWS,street_designation_PATH,street_designation_PIKE
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
9,0,0,0
...,...,...,...
580547,0,0,0
580637,0,0,0
580840,0,0,0
580978,0,0,0


In [46]:
xg_reg.fit(train.drop(['sale_price'], axis=1),
            train['sale_price'])

preds = xg_reg.predict(train.drop(['sale_price'], axis=1))

[14:49:50] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:49:50] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


In [47]:
from sklearn.metrics import mean_squared_error
mean_squared_error(train['sale_price'], preds)

66349189183.271286

We then pickle the dataset for later use and easy loading:    

In [ ]:
pickle_out = open("housedat.pickle","wb")
pickle.dump(dat, pickle_out)
pickle_out.close()

possible predictions datasets:   
https://www.kaggle.com/datasets/harlfoxem/housesalesprediction     
https://github.com/michellesklee/predicting_home_values    